This notebook creates several masking arrays based on [Cropland Data Layer (CDL) classification](https://www.nass.usda.gov/Research_and_Science/Cropland/docs/CDL_codes_names_colors.xls). It assumes that you have already resampled CDL GeoTIFFs to radar coordinates using [ungeocode_to_radar.ipynb](https://github.com/kdevlin525/C-band-phase-bias/blob/main/ungeocode_to_radar.ipynb).

In [8]:
import os, glob
from osgeo import gdal
import math
import numpy as np
from scipy import signal
from scipy.ndimage import label
import cv2

In [9]:
# set path of your working directory
workdir = 'path/to/your/dir' 

# size of entire SLC
az0 = 0
daz = 3500
rg0 = 0
drg = 30000

In [ ]:
# This assumes you already have two files in radar coordinates
# median amplitude of all SLCs
ds = gdal.Open(f'{workdir}/ampmed.r4', gdal.GA_ReadOnly)
ampmed = ds.GetRasterBand(1).ReadAsArray()
# phase stability of all SLCs
ds = gdal.Open(f'{workdir}/phsstab.r4', gdal.GA_ReadOnly)
phsstab = ds.GetRasterBand(1).ReadAsArray() # you won't need this

In [ ]:
# apply thresholds to remove dark and noisy pixels
ampstabarray = np.ones(([daz,drg]))
for i in range(0,daz):
    for j in range(0,drg):
        if np.logical_and(ampmed[i,j]<50,phsstab[i,j]<0.4):
            ampstabarray[i,j] = 0

In [ ]:
# Read in all avaialble resampled data CDL TIFFs
CDLdates=np.arange(2019,2021,1)
CDLnd = len(CDLdates)
allcrops = np.ndarray([daz,drg,CDLnd])

for k in np.arange(CDLnd): #loop through subsequent SLC pairs
    ds = gdal.Open(f'{workdir}/radarfullres/CDL_{str(CDLdates[k])}.tif', gdal.GA_ReadOnly)
    allcrops[:,:,k] = ds.GetRasterBand(1).ReadAsArray(rg0,az0,drg,daz)

roadarray0 = np.zeros(([daz,drg,CDLnd]))
roadarray0[(allcrops>120)&(allcrops<125)] = 1 # all pixels labeled some type of "developed"
roadarray0[allcrops==131] = 1 # all pixels labeled "Barren"
roadarray0[allcrops==65]=1 # another label for "Barren"

# include all pixels always labeled "Fallow"
allfallow=np.zeros(([daz,drg,CDLnd]))
allfallow[allcrops==61]=1
allfallow=np.sum(allfallow,2)
alwaysfallow=np.zeros(([daz,drg]))
alwaysfallow[allfallow==CDLnd]=1

# connstruct final array of all stable pixels
roadarray1=np.sum(roadarray0,2)
roadarray1[alwaysfallow==1]=1
roadarray = (roadarray1>0)*1.0

# mask based on radar metric thresholds
roadarray = roadarray*ampstabarray

# write to a file
driver=gdal.GetDriverByName('ISCE')
colds = driver.Create(f'{workdir}/roadarray.r4',drg,daz,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(roadarray)
colds=None

In [ ]:
# create arrays for each of six crops: almonds, cotton, grapes, pistachios, tomatoes, and winter wheat and save to files

sardates=np.arange(2019,2021,1)
sarnd = len(sardates)

for k in np.arange(sarnd): #loop through each CDL TIFF
    ds = gdal.Open(f'{workdir}/radarfullres/CDL_{str(sardates[k])}.tif', gdal.GA_ReadOnly)
    crops = ds.GetRasterBand(1).ReadAsArray(rg0,az0,drg,daz)
    # erode and dilate kernel
    ckernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,5))
    # almond array
    almdarray = np.zeros(([daz,drg]))
    almdarray[crops==75] = 1
    eroded = cv2.erode(almdarray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    driver=gdal.GetDriverByName('ISCE')
    colds = driver.Create(f'{workdir}/almd{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None
    print(str(k)+' almond done')
    # cotton array
    cottarray = np.zeros(([daz,drg]))
    cottarray[crops==2] = 1
    eroded = cv2.erode(cottarray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    colds = driver.Create(f'{workdir}/cott{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None
    print(str(k)+' cotton done')
    # grape array
    grapearray = np.zeros(([daz,drg]))
    grapearray[crops==69] = 1
    eroded = cv2.erode(grapearray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    colds = driver.Create(f'{workdir}/grape{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None
    print(str(k)+' grape done')
    # pistachio array
    pstcharray = np.zeros(([daz,drg]))
    pstcharray[crops==204] = 1
    eroded = cv2.erode(pstcharray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    colds = driver.Create(f'{workdir}/pstch{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None 
    print(str(k)+' pistachio done')
    # tomato array
    tomarray = np.zeros(([daz,drg]))
    tomarray[crops==54] = 1
    eroded = cv2.erode(tomarray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    colds = driver.Create(f'{workdir}/tom{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None
    print(str(k)+' tomato done')  
    # wheat array
    whtarray = np.zeros(([daz,drg]))
    whtarray[crops==24] = 1
    eroded = cv2.erode(whtarray,ckernel,iterations=7)
    dilated = cv2.dilate(eroded,ckernel,iterations=6)
    dilated = dilated*ampstabarray
    colds = driver.Create(f'{workdir}/wht{str(sardates[k])}array.r4',drg,daz,1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(dilated)
    colds=None
    print(str(k)+' wheat done')


In [ ]:
# read in all crop arrays, combine them, and number each indidivual field

# 2019
ds = gdal.Open(f'{workdir}/almd2019array.r4', gdal.GA_ReadOnly)
almd2019array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/cott2019array.r4', gdal.GA_ReadOnly)
cott2019array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/grape2019array.r4', gdal.GA_ReadOnly)
grape2019array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/pstch2019array.r4', gdal.GA_ReadOnly)
pstch2019array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/tom2019array.r4', gdal.GA_ReadOnly)
tom2019array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/wht2019array.r4', gdal.GA_ReadOnly)
wht2019array = ds.ReadAsArray()
array2019 = almd2019array+cott2019array+grape2019array+pstch2019array+tom2019array+wht2019array

# 2020
ds = gdal.Open(f'{workdir}/almd2020array.r4', gdal.GA_ReadOnly)
almd2020array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/cott2020array.r4', gdal.GA_ReadOnly)
cott2020array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/grape2020array.r4', gdal.GA_ReadOnly)
grape2020array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/pstch2020array.r4', gdal.GA_ReadOnly)
pstch2020array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/tom2020array.r4', gdal.GA_ReadOnly)
tom2020array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/wht2020array.r4', gdal.GA_ReadOnly)
wht2020array = ds.ReadAsArray()
array2020 = almd2020array+cott2020array+grape2020array+pstch2020array+tom2020array+wht2020array

# 2021
ds = gdal.Open(f'{workdir}/almd2021array.r4', gdal.GA_ReadOnly)
almd2021array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/cott2021array.r4', gdal.GA_ReadOnly)
cott2021array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/grape2021array.r4', gdal.GA_ReadOnly)
grape2021array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/pstch2021array.r4', gdal.GA_ReadOnly)
pstch2021array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/tom2021array.r4', gdal.GA_ReadOnly)
tom2021array = ds.ReadAsArray()
ds = gdal.Open(f'{workdir}/wht2021array.r4', gdal.GA_ReadOnly)
wht2021array = ds.ReadAsArray()
array2021 = almd2021array+cott2021array+grape2021array+pstch2021array+tom2021array+wht2021array

# combine into one array of all fields
fieldarray = array2019+array2020+array2021
fieldarray[fieldarray>1] = 1

# set any overlapping pixels between fields and stable arrays as 0
fieldarray[roadarray==1] = 0

In [ ]:
# count number of individual features and label 
structure = np.ones((3, 3), dtype=np.int_) 
labeled, ncomponents = label(fieldarray, structure)

# write to file
driver=gdal.GetDriverByName('ISCE')
colds = driver.Create(f'{workdir}/fieldccarray.r4',drg,daz,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(labeled)
colds=None